# Librerías

In [21]:
import os
import time
import re
import chromadb
from chromadb.utils import embedding_functions

# Constantes


In [22]:
# Ruta del documento
DOC_PATH = "./test.txt"

# Directorio de la base de datos Chroma
CHROMA_DIR = "./chroma_db"

# Nombre de la colección
COLLECTION_NAME = "mini_rag_docs"

# Tamaño de los chunks
CHUNK_SIZE = 900

# Solapamiento entre chunks
OVERLAP = 200      

# Leer documento

In [23]:
def read_txt(path: str, encoding: str = "utf-8") -> str:
    with open(path, "r", encoding=encoding, errors="ignore") as f:
        return f.read()

def load_document(path: str) -> tuple[str, dict]:
    if not os.path.exists(path):
        raise FileNotFoundError(f"No existe: {path}")
    text = read_txt(path)
    meta = {
        "source": os.path.basename(path),
        "path": os.path.abspath(path),
        "loaded_at": time.strftime("%Y-%m-%d %H:%M:%S"),
    }
    return text, meta


# Limpieza del texto


In [24]:
def clean_text_basic(text: str) -> str:
    text = text.replace("\r\n", "\n").replace("\r", "\n")
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()


def chunk_text(text: str, chunk_size: int = 800, overlap: int = 150) -> list[str]:
    if chunk_size <= 0:
        raise ValueError("chunk_size debe ser > 0")
    if overlap < 0:
        raise ValueError("overlap debe ser >= 0")
    if overlap >= chunk_size:
        raise ValueError("overlap debe ser < chunk_size")

    chunks = []
    start = 0
    n = len(text)
    while start < n:
        end = min(start + chunk_size, n)
        chunk = text[start:end].strip()
        if chunk:
            chunks.append(chunk)
        if end == n:
            break
        start = end - overlap
    return chunks


# Crear indice en Chroma

In [25]:
# Embeddings locales
ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"  # 384 dimensiones
)

# Cliente Chroma
client = chromadb.PersistentClient(path=CHROMA_DIR)


def get_collection(name: str):
    return client.get_or_create_collection(
        name=name,
        metadata={"hnsw:space": "cosine"},
        embedding_function=ef,
    )


def reset_collection(name: str):
    try:
        client.delete_collection(name)
    except Exception:
        pass
    return get_collection(name)

In [26]:
def build_chroma_index(doc_path: str, collection_name: str, chunk_size: int, overlap: int, reset: bool = True):
    # Cargar
    raw_text, doc_meta = load_document(doc_path)
    text = clean_text_basic(raw_text)

    # Trocear
    chunks = chunk_text(text, chunk_size=chunk_size, overlap=overlap)

    # Colección
    col = reset_collection(collection_name) if reset else get_collection(collection_name)

    # Insertar
    ids = [f"{doc_meta['source']}::chunk_{i}" for i in range(len(chunks))]
    metadatas = []
    for i in range(len(chunks)):
        metadatas.append({
            "source": doc_meta["source"],
            "chunk_index": i,
            "chunk_size": chunk_size,
            "overlap": overlap,
        })

    # Añadir a la colección
    col.add(ids=ids, documents=chunks, metadatas=metadatas)

    info = {
        "doc": doc_meta,
        "n_chunks": len(chunks),
        "collection": collection_name,
        "chunk_size": chunk_size,
        "overlap": overlap,
        "built_at": time.strftime("%Y-%m-%d %H:%M:%S"),
        "persist_dir": os.path.abspath(CHROMA_DIR),
    }
    return col, info


In [27]:
collection, build_info = build_chroma_index(
    doc_path=DOC_PATH,
    collection_name=COLLECTION_NAME,
    chunk_size=CHUNK_SIZE,
    overlap=OVERLAP,
    reset=True,
)

build_info

{'doc': {'source': 'test.txt',
  'path': '/home/juan/BBDD/EjercicioRAG/test.txt',
  'loaded_at': '2026-01-20 19:45:17'},
 'n_chunks': 7,
 'collection': 'mini_rag_docs',
 'chunk_size': 900,
 'overlap': 200,
 'built_at': '2026-01-20 19:45:18',
 'persist_dir': '/home/juan/BBDD/EjercicioRAG/chroma_db'}

# Consultas


In [28]:
def ask(query: str, top_k: int = 5, show_chars: int = 350):
    res = collection.query(query_texts=[query], n_results=top_k)

    docs = res.get("documents", [[]])[0]
    metas = res.get("metadatas", [[]])[0]
    dists = res.get("distances", [[]])[0]

    print("PREGUNTA:", query)
    print("=" * 90)

    for i, (doc, meta, dist) in enumerate(zip(docs, metas, dists), start=1):
        sim = None
        if dist is not None:
            sim = 1 - float(dist)
        src = meta.get("source") if isinstance(meta, dict) else None
        idx = meta.get("chunk_index") if isinstance(meta, dict) else None

        line = f"[{i}] sim={sim:.4f}  dist={dist:.4f}  source={src}  chunk={idx}"
        print(line)
        snippet = (doc or "")[:show_chars].replace("\n", " ").replace("\r", " ")
        print(" ", snippet + (" ..." if (doc and len(doc) > show_chars) else ""))
        print("-" * 90)

In [29]:
ask("¿De qué trata el documento?")

PREGUNTA: ¿De qué trata el documento?
[1] sim=0.6014  dist=0.3986  source=test.txt  chunk=2
  n numérica (vector) de un texto. Se genera usando un modelo entrenado, por ejemplo Sentence Transformers. La idea es que textos con significado similar tengan embeddings similares.  Ejemplo: - "¿Qué es Chroma?" y "Explícame la librería Chroma DB" deberían producir embeddings cercanos. - "Cómo cocinar arroz" debería estar lejos de "optimización de b ...
------------------------------------------------------------------------------------------
[2] sim=0.5271  dist=0.4729  source=test.txt  chunk=0
  TEMA: INTRODUCCIÓN A SISTEMAS DE RECUPERACIÓN DE INFORMACIÓN Y RAG  1. ¿Qué es un sistema de Recuperación de Información? Un sistema de Recuperación de Información (Information Retrieval, IR) es un conjunto de técnicas y herramientas que permiten buscar información relevante dentro de una colección de documentos. A diferencia de una búsqueda por co ...
-------------------------------------------------

In [30]:
ask("¿Qué significa RAG?")

PREGUNTA: ¿Qué significa RAG?
[1] sim=0.5045  dist=0.4955  source=test.txt  chunk=6
  os más relevantes con ciertos parámetros.  Ejemplos de preguntas recomendadas: - ¿Qué es RAG? - ¿Qué es un embedding? - ¿Para qué sirve una base de datos vectorial? - ¿Qué parámetros se ajustan en el chunking? - ¿Qué hace Chroma en un sistema RAG?  9. Conclusión Un mini sistema RAG permite construir una base sólida para aplicaciones más avanzadas c ...
------------------------------------------------------------------------------------------
[2] sim=0.4611  dist=0.5389  source=test.txt  chunk=2
  n numérica (vector) de un texto. Se genera usando un modelo entrenado, por ejemplo Sentence Transformers. La idea es que textos con significado similar tengan embeddings similares.  Ejemplo: - "¿Qué es Chroma?" y "Explícame la librería Chroma DB" deberían producir embeddings cercanos. - "Cómo cocinar arroz" debería estar lejos de "optimización de b ...
---------------------------------------------------------